In [1]:
import pandas as pd
import math

### Functions

In [5]:
def add_line_2_line_df(df_lines_bc):
    '''
    This function add lines to the dataset which are missing.
    Currently only a connection between BC_WAX_GSS and BC_WHS_TSS is missing.
    '''
    # Add line connecting BC_WAX_GSS
    data = df_lines_bc[(df_lines_bc["ending_node_code"].str.contains("WAN")) & (df_lines_bc["circuit_id"].str.contains("14L"))]
    if data.shape[0] == 1:
        row = data.iloc[0].copy()
        row.loc["ending_node_code"] = "BC_WAX_GSS" # edit name
        df_lines_bc.loc[df_lines_bc.index.max() + 1] = row
    else:
        print(f'Did not index BC_WAN_GSS wiht circuit id 14L') # logging

def correct_line_node_name(df_lines):
    ''' 
    This function corrects nodes in the lines dataset from CODERS which were improperly named.
    '''
    
    node_cols = ["starting_node_code", "ending_node_code"]
    old_2_new = {"BC_WHO_JCT":"BC_WAH_JCT",
                 "BC_GST_JCT":"BC_MCK_JCT",
                 "BC_VSY_JCT":"BC_NOR_DSS"}
    for col in node_cols:
        for old_node,new_node in old_2_new.items():
            df_lines.loc[df_lines[col] == old_node, col] = new_node

def check_missing_buses(df_sub_bc, df_lines_bc):
    '''
    Checks for substations which are missing from the lines dataset.
    The only bus like this for BC is BC_WAX_GSS which should be connceted to BC_WAN.
    df_buses_bc: DF being prepared for saving and loading into PyPSA.
    df_sub_bc: DF from CODERS of all substations.
    '''
    # Checking to BC_WAX_GSS is the only one missing
    sub_unique_codes = df_sub_bc["node_code"].apply(lambda x: x.split('_')[1].strip(' ')).unique().tolist()
    line_unique_codes_start = df_lines_bc["starting_node_code"].apply(lambda x: x.split('_')[1].strip(' ')).unique().tolist()
    line_unique_codes_end = df_lines_bc["ending_node_code"].apply(lambda x: x.split('_')[1].strip(' ')).unique().tolist()
    line_unique_codes = list(set(line_unique_codes_start + line_unique_codes_end))

    for code in sub_unique_codes:
        if code not in line_unique_codes:
            print(f"the code {code} is missing from the lines dataset!") # to be logged

def add_pypsa_columns_2_line_df(df):
    ''' 
    This function will add the columns to the line df which will be imported into a pypsa network.
    line names assigned according to standard of voltage (i.e. 230_AAL, 230 = 230kV and AAL = middle 3 char of node_code).
    '''
    name = []
    type = []
    bus0 = []
    bus1 = []
    length = []
    v_nom = []

    for idx,line in df.iterrows():
        voltage_type = f'{str(int(line["voltage_in_kv"]))}kV'
        name.append(line["starting_node_code"][3:] + line["ending_node_code"][2:]) # XXX_GSS_YYY_DSS
        type.append(voltage_type)
        bus0.append(voltage_type.rstrip('kV') + "_" + line['starting_node_code'].split('_')[1] + "_" + line['starting_node_code'].split('_')[2])
        bus1.append(voltage_type.rstrip('kV') + "_" + line['ending_node_code'].split('_')[1] + "_" + line['ending_node_code'].split('_')[2])
        length.append(line["line_segment_length_km"])
        v_nom.append(line["voltage_in_kv"])

    df['name'] = name
    df['type'] = type
    df['bus0'] = bus0
    df['bus1'] = bus1
    df['length'] = length
    df['v_nom'] = v_nom

def create_bus_df(df_lines, df_substations):
    '''
    This function will create an initial DF buses for PyPSA from a DF of lines.
    When creating the buses it
    The lines DF contains the node names for the buses, nomial voltage, and the carrier is implicitly added.
    '''
    # name = []
    # x = []
    # y = []
    # type = []
    # v_nom = []
    data_dict = {}

    # (1) Add buses based on line nodes
    for idx,line in df_lines.iterrows():
        # Search for match between line and substation
        for node_code in [line["starting_node_code"], line["ending_node_code"]]:
            bus_name, bus_x, bus_y = get_bus_name_x_y(line, node_code, df_substations)
            if bus_name not in data_dict: # Avoid duplication (Change to dictionary)
                data_dict[bus_name] = {'x':bus_x, 'y':bus_y, 'type':line['type'], 'v_nom':line['v_nom']}
                # name.append(bus_name) # i.e. 230_AAL
                # x.append(bus_x)
                # y.append(bus_y)
                # type.append(line['type'])
                # v_nom.append(line['v_nom'])
    
    df_buses = pd.DataFrame.from_dict(data_dict,orient='index').reset_index().rename(columns={'index':'name'})
    # df_buses = pd.DataFrame()
    # df_buses = pd.DataFrame()
    # df_buses['name'] = name
    # df_buses['x'] = x
    # df_buses['y'] = y
    # df_buses['type'] = type
    # df_buses['v_nom'] = v_nom

    return df_buses

def get_bus_name_x_y(line, node_code, df_substations):
    '''
    This function finds the correct name for a bus from the node dataset
    line: Line row from CODERS lines df.
    node_code: Name of start/end node from CODERS lines df.
    df_substations: Substations df from CODERS.
    return bus_name: Unique name to use for the bus in PyPSA (i.e. 230_AAL_DSS, 230=nominal_voltage, AAL=unique substation name in CODERS, DSS=Substation type)
    return bus_x
    '''
    # (1) identical match
    for idx,substation in df_substations.iterrows():
        if substation["node_code"] == node_code:
            bus_name = str(line["v_nom"]) + "_" + "_".join(node_code.split('_')[1:]) # (i.e. 230_AAL_DSS)
            bus_x = substation["longitude"]
            bus_y = substation["latitude"]
            return bus_name, bus_x, bus_y

    # print(f"Did not find exact match for line node: {node_code}") # To-be logged

    # (2) International and Interprovincial nodes
    if node_code.split('_')[-1] in ["IPT","INT"]:
        bus_name = str(line["v_nom"]) + "_" + "_".join(node_code.split('_')[1:])
        if node_code == "PP_BCAB3_IPT":
            # ~ 50 km east
            bus_y, bus_x = 50.247937, -114.2 
        elif node_code == "PP_BCAB1_IPT":
            # ~ 18 km east
            bus_y, bus_x = 49.735535, -114.6
        elif node_code == "PP_BCAB4_IPT":
            # ~21 km east
            bus_y, bus_x = 58.64525, -119.7
        elif node_code == "XX_BCUS2_INT":
            # ~ 1 km south
            bus_y, bus_x = 48.9974, -117.341514
        elif node_code == "XX_BCUS1_INT":
            # ~ 21 km south
            bus_y, bus_x = 48.97 , -122.873948
        elif node_code == "PP_BCAB2_IPT":
            # ~ 108 km east
            bus_y, bus_x = 49.500543, -114.08
        return bus_name, bus_x, bus_y

    # (3) Find first matching 3-middle characters
    for idx,substation in df_substations.iterrows():
        if node_code.split('_')[1] == substation['node_code'].split('_')[1]:
            bus_name = str(line["v_nom"]) + "_" + "_".join(node_code.split('_')[1:])
            bus_x = substation["longitude"]
            bus_y = substation["latitude"]
            return bus_name, bus_x, bus_y
    # print(f"Did not find partial match for: {node_code}") # To-be logged

    # (4) No matching 3 middle characters (i.e. BC_WAX_GSS).. these are special cases..
    print(f"There is no information to create bus for: {node_code}") # To-be logged

    return None,None,None

def create_line_types_df(df_lines, df_line_table):
    '''
    This function adds line type information for each line.
    Assuming all lines can have their transmission inferred on the basis of ampacity alone...
    # eventually will needs a calculator based on short, medium, or long and voltage.
    r: resistance per length (Ohm per km)
    x: resistance/reactance per length (Ohm per km)
    c: shunt capacitance per length (nF per km)
    i: Nominal current (kA)
    cc: Cross section (mm^2)
    1) match based on closest match in the table
    2) match based on average for similar lines???
    '''
    ampacity_sel_col = "summer_ampacity"
    data_dict = {'name':[],
                'f_nom':[],
                'r_per_length':[],
                'x_per_length':[],
                'c_per_length':[],
                'i_nom':[],
                'mounting':[],
                'cross_section':[]}
    line_type_col = []
    amp_cap_2_idx = {amp_cap:idx for idx,amp_cap in enumerate(df_line_table["approx_current_capacity"])}

    for _,row in df_lines.iterrows():
        ampacity = row[ampacity_sel_col] # later can make this variable as needed
        if not math.isnan(ampacity):
            name = str(int(ampacity))
            line_type_col.append(name) # add line type for matching within PyPSA

            if name not in data_dict["name"]:
                idx = sorted([(abs(amp_cap-ampacity), idx) for amp_cap,idx in amp_cap_2_idx.items()])[0][-1] # Find row index for closest ampacity in the table
                data_dict["name"].append(name) # more descriptive name later (ampacity for now)
                data_dict["f_nom"].append(60)
                data_dict['r_per_length'].append(df_line_table["resistance_ac_25_deg"].iloc[idx] / 1000)
                data_dict['x_per_length'].append(df_line_table["x_l"].iloc[idx])
                data_dict['c_per_length'].append(8.85) # assumed based on VI-PyPSA
                data_dict['i_nom'].append(df_line_table["resistance_ac_25_deg"].iloc[idx] / 1000)
                data_dict['mounting'].append("ol")
                data_dict['cross_section'].append(df_line_table["cross_section_mm2"].iloc[idx] )

            else:
                continue # already have this line type added
        else:
            # use mode ampacity for same voltage type 
            if row["voltage_in_kv"] == 63:# replace 63 kV since no ampacity on it
                voltage = 69 
            elif row["voltage_in_kv"] == 161: # replace 161 kV since no ampacity for it
                voltage = 138 
            else:
                voltage = row["voltage_in_kv"]

            ampacity = df_lines[(df_lines["voltage_in_kv"] == voltage) & (~df_lines["summer_ampacity"].isnull())]["summer_ampacity"].mode()[0]
                
            line_type_col.append(str(int(ampacity)))

    df_lines["type"] = line_type_col
    df_line_types = pd.DataFrame(data_dict)

    return df_line_types

def rename_duplicate_lines(df_lines_bc):
    '''
    Renamed duplicate lines, adding _# to reach in order found.
    '''
    indices = df_lines_bc[df_lines_bc.duplicated(subset=["name"],keep=False)].index.tolist()
    name_dict = {} # keep record of modifications
    for index in indices:
        name = df_lines_bc.loc[index,"name"]
        if name not in name_dict.keys():      
            name_dict[name] = 1
        else:
            name_dict[name] += 1
        df_lines_bc.loc[index,"name"] = name + "_" + str(name_dict[name])

def add_line_op_params(df_lines_bc, df_line_types_bc):
    '''
    This function add operational parameters to the lines such as:
    s_nom = voltage * i_nom
    '''
    df_lines_bc['s_nom'] = df_lines_bc.apply(lambda line: add_line_s_nom(line),axis=1)

def add_line_s_nom(line):
    '''
    This function is applied row-wise to calculate the s_nom (MVA) for each line.
    Using data from CODERS to impute the s_nom value for lines as follows:
    option 1: use summer_rating_in_mva
    option 2: use summer_capacity_in_mw adjusted by a power factor = 0.9 
    Using the summer values is a pessimistic assumption for other seasons.
    '''
    if line['summer_rating_in_mva'] == 0:
        s_nom = round(line['summer_capacity_in_mw'] / 0.9, 4) # (assumed 0.9 power factor + rounded to 4th decimal)
    else:
        s_nom = line['summer_rating_in_mva']

    return s_nom

### Calls

In [6]:
# load data
transmission_line_path = r"C:\Users\pmcw9\Delta-E\PICS\Data\SESIT\CODERS\data-pull\network\transmission_lines.csv"
substations_path = r"C:\Users\pmcw9\Delta-E\PICS\Data\SESIT\CODERS\data-pull\network\substations.csv"
transmission_line_type_table = r"C:\Users\pmcw9\Delta-E\PICS\Data\custom\electric_power_generation_table_13_3a.xlsx"

df_lines = pd.read_csv(transmission_line_path)
df_substations = pd.read_csv(substations_path)
df_line_table = pd.read_excel(transmission_line_type_table) # Tables with line type data for indexing by ampacities

df_sub_bc = df_substations[df_substations["province"] == "BC"].copy()
df_lines_bc = df_lines[df_lines["province"] == "BC"].copy()

In [7]:
# BC_WAX_GSS is only substation which is not in the line dataset. I will need to look into implications of this when supply connected.
# (0A) Correction to data
correct_line_node_name(df_lines_bc) 
df_sub_bc["node_code"] = df_sub_bc["node_code"].apply(lambda x: x.replace(" ",""))
df_lines_bc["starting_node_code"] = df_lines_bc["starting_node_code"].apply(lambda x: x.replace(" ",""))
df_lines_bc["ending_node_code"] = df_lines_bc["ending_node_code"].apply(lambda x: x.replace(" ",""))
df_lines_bc['summer_rating_in_mva'] = df_lines_bc["summer_rating_in_mva"].fillna(0.)

# (0B) Add missing lines to dataset
add_line_2_line_df(df_lines_bc)

# (1) Enrich coders dataframe of BC lines with columns used by PyPSA 
add_pypsa_columns_2_line_df(df_lines_bc)

# (2) Create dataframe of BC buses from the lines and substations
df_buses_bc = create_bus_df(df_lines_bc, df_sub_bc) 
check_missing_buses(df_sub_bc, df_lines_bc)

# (3) create dataframe of line types for BC
df_line_types_bc = create_line_types_df(df_lines_bc, df_line_table)

# (4) rename lines which are duplicates (add suffix of _#)
rename_duplicate_lines(df_lines_bc)

# (5) add all needed operational parameters to lines
add_line_op_params(df_lines_bc, df_line_types_bc)

# (6) create dataframe of transformers for BC
# df_transformers_bc = ..

# (7) create dataframe of transformer types for BC
# df_transformer_types_bc = ..

## Additional attributes
df_buses_bc['node_type'] = df_buses_bc['name'].apply(lambda x: x.split('_')[-1])

In [8]:
# Writing to folder
# 138_KWL_GSS missing from buses dataframe
path = r"C:\Users\pmcw9\Delta-E\PICS\PyPSA_BC\results\pypsa-network"

df_lines_bc.to_csv(path + "\\lines.csv", index=False, columns=['name','type','bus0','bus1','length','v_nom','s_nom'])
df_buses_bc.to_csv(path + "\\buses.csv", index=False)
df_line_types_bc.to_csv(path + "\\line_types.csv", index=False)

In [ ]:
df_buses_bc[df_buses_bc['name'].str.contains('VIT')]

,name,x,y,type,v_nom,node_type
13,138_VIT_TSS,-123.716421,48.82734,138kV,138,TSS
378,230_VIT_TSS,-123.716421,48.82734,230kV,230,TSS


In [ ]:
# leave = False
# for idx,line in df_lines_bc.iterrows():
#     type = line['type']
#     line_type_matched = df_line_types_bc[df_line_types_bc['name'] == type]
#     if line_type_matched.shape[0] != 1:
#         print('Error incorrect number of matches for line types.')
#         break
#     i_nom = line_type_matched['i_nom'].iloc[0]
#     option_1 = i_nom * line['v_nom']
#     option_2 = option_1 * line['length']
#     if line['summer_rating_in_mva'] == 0:
#         option_3 = round(line['summer_capacity_in_mw'] / 0.9, 4) # (assumed 0.9 power factor + rounded to 4th decimal)
#     else:
#         option_3 = line['summer_rating_in_mva']
#         leave = True
#     print('-------------------------') 
#     print(f'The estimated s_nom (MVA) based on i_nom * v_nom = {option_1} MVA')
#     print(f'The estimated s_nom (MVA) based on i_nom * v_nom * length = {option_2} MVA')
#     print(f'The estimated s_nom (MVA) based on CODERS is {option_3} MVA')

#     if leave:
#         break